# TP4 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

# Enunciado do Problema 2

O programa `Python` seguinte implementa o algoritmo de bubble sort para ordenação in situ de um array de inteiros `seq`.

In [1]:
seq = [-2,1,2,-1,4,-4,-3,3]
changed = True
while changed:
    changed = False
    for i in range(len(seq) - 1):
        if seq[i] > seq[i+1]:
            seq[i], seq[i+1] = seq[i+1], seq[i]
            changed = True
print(seq)

[-4, -3, -2, -1, 1, 2, 3, 4]


 1. Defina a pré-condição e a pós-condição que descrevem a especificação deste algoritmo.
 2. O ciclo `for` pode ser descrito por uma transição $\,\mathtt{seq}\gets \mathit{exp}(\mathtt{seq})\,$. Construa uma relação de transição $\;\mathsf{trans}(\mathtt{seq},\mathtt{seq}')\,$ que modele esta atribuição.
 3. Usando a técnica que lhe parecer mais conveniente verifique a correção do algoritmo.

```python
    Pré-Condição: assume n >= 0 and changed == True
    Pós-Condução: assert forall j . 0 <= j < n-1 -> seq[j] <= seq[j+1] and changed == False
```

In [2]:
from pysmt.shortcuts import *
from pysmt.typing import *

In [3]:
n = Symbol('n',INT)
i = Symbol('i', INT)
seq = Symbol('seq', ArrayType(INT, INT))

In [4]:
f = Symbol('f', FunctionType(INT,[INT]))
ax1 = Equals(f(Int(0)), Select(seq, Int(0)))
ax2 = ForAll([i], Implies(And(i>Int(0), i<n), Equals(f(i), Ite(Select(seq,i) >= f(i-Int(1)), Select(seq,i), f(i-Int(1))))))
axioms = And(ax1,ax2)

In [5]:
def for_cycle(c, p, changed, changed_p):
    print(c,p)
    indutivo = ForAll([i], Implies(And(i<n-Int(1), i>=Int(0)), 
                                   Equals(Select(p,i), Ite(Select(c, i+Int(1)) <= f(i), Select(c, i+Int(1)), f(i)))))
    
    final = Equals(Select(p, n - Int(1)), f(n - Int(1)))
    
    #troca_true = Bool(True)
    
    #troca_false = Bool(True)
    
    #troca_true  = Iff(Iff(changed_p, Bool(True)), Exists([i], Implies(And(i>=Int(0), i<n), Not(Equals(Select(c, i), Select(p, i))))))
    
    #troca_false = Iff(Iff(changed_p, Bool(False)), ForAll([i], Implies(And(i>=Int(0), i<n), Equals(Select(c, i), Select(p, i))))) 
    
    troca_true = Iff(Iff(changed_p, Bool(True)), Not(Equals(c,p)))
    
    troca_false = Iff(Iff(changed_p, Bool(False)), Equals(c,p))
    
    return And(axioms,indutivo, troca_true, troca_false, final)

In [65]:
# Auxiliares
def prime(v):
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())
def fresh(v):
    return FreshSymbol(typename=v.symbol_type(),template=v.symbol_name()+"_%d")

# A classe "Sigle Assignment Unfold"
class SAU(object):
    """Trivial representation of a while cycle and its unfolding."""
    def __init__(self, variables, pre , pos, control, trans, sname="z3"):
              
        self.variables = variables       # variables   
        self.pre = pre                   # pre-condition as a predicate in "variables"
        self.pos = pos                   # pos-condition as a predicate in "variables"
        self.control = control           # cycle control as a predicate in "variables"
        self.trans = trans               # cycle body as a binary transition relation 
                                         # in "variables" and "prime variables"
        
        self.prime_variables = [prime(v) for v in self.variables]
        self.frames = [And([Not(control),pos])]
                 # inializa com uma só frame: a da terminação do ciclo
        
        self.solver = Solver(name=sname)
        self.fresh = []
    def new_frame(self):        
        freshs = [fresh(v) for v in self.variables]    
        self.fresh = self.fresh + freshs
        b = self.control
        S = self.trans.substitute(dict(zip(self.prime_variables,freshs)))
        print("trans:",S.serialize())
        W = self.frames[-1].substitute(dict(zip(self.variables,freshs)))
        print("w:",W.serialize())
        self.frames.append(And([b , ForAll(freshs, Implies(S, W))]))
        
    def unfold(self,bound=0):
        n = 0
        while True:
            if n > bound:
                print("falha: número de tentativas ultrapassa o limite %d "%bound)
                break
            
            f = Or(self.frames)
            if self.solver.solve([self.pre,Not(f)]):
                print(self.solver.get_value(changed))
                print("seq:")
                for inc in range(N):
                    print(self.solver.get_value(Select(seq, Int(inc))))
                print("freshs:")
                for v in self.fresh:
                    if str(v.symbol_type()) == "Array{Int, Int}":
                        for inc in range(N):
                            print(v.symbol_name(), inc, ":", self.solver.get_value(Select(v, Int(inc))))
                print("end frame")
                self.new_frame()
                n += 1
            else:
                print("sucesso na tentativa %d "%n)
                break   

In [66]:
N = 8
n_s = Int(N)

store1 = And(Equals(Select(seq, Int(0)), Int(-2) ),
            Equals(Select(seq, Int(1)), Int(1)  ),
            Equals(Select(seq, Int(2)), Int(2)  ),
            Equals(Select(seq, Int(3)), Int(-1) ),
            Equals(Select(seq, Int(4)), Int(4)  ),
            Equals(Select(seq, Int(5)), Int(-4) ),
            Equals(Select(seq, Int(6)), Int(-3) ),
            Equals(Select(seq, Int(7)), Int(3)  ))

changed = Symbol('changed', BOOL)
variables = [changed, seq]
print(seq.symbol_type())

Array{Int, Int}


In [67]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store1)  
pos  =  And(ForAll([i], Implies(And(i>=0, i<n_s-1), Select(seq, i) <= Select(seq, i+1))), Iff(changed, Bool(False)))
cond =  Iff(changed, Bool(True))               # condição de controlo do ciclo
trans = for_cycle(seq, prime(seq), changed, prime(changed))

W = SAU(variables, pre, pos, cond, trans)
W.unfold(8)

seq 'next(seq)'
True
seq:
-2
1
2
-1
4
-4
-3
3
freshs:
end frame
trans: (((f(0) = seq[0]) & (forall i . (((0 < i) & (i < n)) -> (f(i) = ((f((i - 1)) <= seq[i]) ? seq[i] : f((i - 1))))))) & (forall i . (((i < (8 - 1)) & (0 <= i)) -> (seq_209[i] = ((seq[(i + 1)] <= f(i)) ? seq[(i + 1)] : f(i))))) & ((changed_208 <-> True) <-> (! (seq = seq_209))) & ((changed_208 <-> False) <-> (seq = seq_209)) & (seq_209[(8 - 1)] = f((8 - 1))))
w: ((! (changed_208 <-> True)) & ((forall i . (((0 <= i) & (i < (8 - 1))) -> (seq_209[i] <= seq_209[(i + 1)]))) & (changed_208 <-> False)))
True
seq:
-2
1
2
-1
4
-4
-3
3
freshs:
seq_209 0 : 0
seq_209 1 : 0
seq_209 2 : 0
seq_209 3 : 0
seq_209 4 : 0
seq_209 5 : 0
seq_209 6 : 0
seq_209 7 : 0
end frame
trans: (((f(0) = seq[0]) & (forall i . (((0 < i) & (i < n)) -> (f(i) = ((f((i - 1)) <= seq[i]) ? seq[i] : f((i - 1))))))) & (forall i . (((i < (8 - 1)) & (0 <= i)) -> (seq_211[i] = ((seq[(i + 1)] <= f(i)) ? seq[(i + 1)] : f(i))))) & ((changed_210 <-> True) <-> (! (seq = 

In [61]:
store2 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N-1)] + [Equals(Select(seq, Int(7)), Int(-3))])
print(store2)
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store2)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -3))
True
seq:
-2
-2
-2
-2
-2
-2
-2
-3
freshs:
end frame
trans: (((f(0) = seq[0]) & (forall i . (((0 < i) & (i < n)) -> (f(i) = ((f((i - 1)) <= seq[i]) ? seq[i] : f((i - 1))))))) & (forall i . (((i < (8 - 1)) & (0 <= i)) -> (seq_181[i] = ((seq[(i + 1)] <= f(i)) ? seq[(i + 1)] : f(i))))) & ((changed_180 <-> True) <-> (! (seq = seq_181))) & ((changed_180 <-> False) <-> (seq = seq_181)) & (seq_181[(8 - 1)] = f((8 - 1))))
True
seq:
-2
-2
-2
-2
-2
-2
-2
-3
freshs:
seq_181 : 0
seq_181 : 0
seq_181 : 0
seq_181 : 0
seq_181 : 0
seq_181 : 0
seq_181 : 0
seq_181 : 0
end frame
trans: (((f(0) = seq[0]) & (forall i . (((0 < i) & (i < n)) -> (f(i) = ((f((i - 1)) <= seq[i]) ? seq[i] : f((i - 1))))))) & (forall i . (((i < (8 - 1)) & (0 <= i)) -> (seq_183[i] = ((seq[(i + 1)] <= f(i)) ? seq[(i + 1)] : f(i))))) & ((changed_182 <-> True) <-> (! (seq = seq_183))) & ((changed_182 <-> False

In [12]:
store3 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N)])
print(store3)
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store3)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -2))
True
True
True
True
True
True
True
True
True
True
falha: número de tentativas ultrapassa o limite 9 


In [21]:
p = prime(seq)
t = for_cycle(seq, p, changed, prime(changed))
with Solver(name="z3") as solver:
    if solver.solve([store3, t, pre]):
        print("Oh")
        #print(solver.get_value(seq))
        for inc in range(N):
            print(solver.get_value(Select(seq, Int(inc))))
        print("diff")
        for inc in range(N):
            print(solver.get_value(Select(p, Int(inc))))
        print(solver.get_value(changed))
        print(solver.get_value(prime(changed)))
    else:
        print("Yo")

seq 'next(seq)'
Oh
-2
-2
-2
-2
-2
-2
-2
-2
diff
-2
-2
-2440
-2
-2
-2
-3
8
True
True


In [ ]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)